# Correlating Returns

In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## Load API Keys from Environment Variables

In [2]:
# Load .env enviroment variables
load_dotenv('example.env')

# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

## Get AAPL Returns for Past Month

In [3]:
# Set the ticker
ticker = "AAPL"

# Set timeframe to '1D'
timeframe = "1D"

# Set current date and the date from one month ago using the ISO format
current_date = pd.Timestamp(datetime.now(), tz="America/New_York").isoformat()
past_date = pd.Timestamp(datetime.now()- timedelta(30), tz="America/New_York").isoformat()

# Get 4 weeks worth of historical data for AAPL
df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=past_date,
    end=current_date,
    after=None,
    until=None,
).df

# Display data
df.head()

AAPL                                     
                             open    high     low     close     volume
time                                                                  
2021-05-10 00:00:00-04:00  129.40  129.54  126.81  126.8600   76030609
2021-05-11 00:00:00-04:00  123.50  126.27  122.77  125.8500  112319323
2021-05-12 00:00:00-04:00  123.52  124.64  122.25  122.8200   94495922
2021-05-13 00:00:00-04:00  124.58  126.15  124.26  124.9401   93465117
2021-05-14 00:00:00-04:00  126.27  127.89  125.85  127.3900   67878566

In [4]:
# Drop Outer Table Level
df = df.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
df = df.drop(columns=["open", "high", "low", "volume"])

# Since this is daily data, we can keep only the date (remove the time) component of the data
df.index = df.index.date

# Display sample data
df.head()

,close
2021-05-10,126.8600
2021-05-11,125.8500
2021-05-12,122.8200
2021-05-13,124.9401
2021-05-14,127.3900


In [5]:
# Use the `pct_change` function to calculate daily returns of AAPL
aapl_returns = df.pct_change().dropna()

# Display sample data
aapl_returns.head()

,close
2021-05-11,-0.007962
2021-05-12,-0.024076
2021-05-13,0.017262
2021-05-14,0.019609
2021-05-17,-0.008870


In [6]:
# Use newsapi client to get most relevant 20 headlines per day in the past month
def get_headlines(keyword):
    all_headlines = []
    all_dates = []    
    date = datetime.strptime(current_date[:10], "%Y-%m-%d")
    end_date = datetime.strptime(past_date[:10], "%Y-%m-%d")
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > end_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date)[:10],
            to=str(date)[:10],
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
    return all_headlines, all_dates

Note: Be aware that running the 3 requests below will only work once within a 24 hour period due to the request limits imposed by the API provider.

In [7]:
# Get first topic
aapl_headlines, dates = get_headlines("apple")

Fetching news about 'apple'
******************************
retrieving news from: 2021-06-08 00:00:00
retrieving news from: 2021-06-07 00:00:00
retrieving news from: 2021-06-06 00:00:00
retrieving news from: 2021-06-05 00:00:00
retrieving news from: 2021-06-04 00:00:00
retrieving news from: 2021-06-03 00:00:00
retrieving news from: 2021-06-02 00:00:00
retrieving news from: 2021-06-01 00:00:00
retrieving news from: 2021-05-31 00:00:00
retrieving news from: 2021-05-30 00:00:00
retrieving news from: 2021-05-29 00:00:00
retrieving news from: 2021-05-28 00:00:00
retrieving news from: 2021-05-27 00:00:00
retrieving news from: 2021-05-26 00:00:00
retrieving news from: 2021-05-25 00:00:00
retrieving news from: 2021-05-24 00:00:00
retrieving news from: 2021-05-23 00:00:00
retrieving news from: 2021-05-22 00:00:00
retrieving news from: 2021-05-21 00:00:00
retrieving news from: 2021-05-20 00:00:00
retrieving news from: 2021-05-19 00:00:00
retrieving news from: 2021-05-18 00:00:00
retrieving news f

In [8]:
# Get second topic
trade_headlines, _ = get_headlines("trade")

Fetching news about 'trade'
******************************
retrieving news from: 2021-06-08 00:00:00
retrieving news from: 2021-06-07 00:00:00
retrieving news from: 2021-06-06 00:00:00
retrieving news from: 2021-06-05 00:00:00
retrieving news from: 2021-06-04 00:00:00
retrieving news from: 2021-06-03 00:00:00
retrieving news from: 2021-06-02 00:00:00
retrieving news from: 2021-06-01 00:00:00
retrieving news from: 2021-05-31 00:00:00
retrieving news from: 2021-05-30 00:00:00
retrieving news from: 2021-05-29 00:00:00
retrieving news from: 2021-05-28 00:00:00
retrieving news from: 2021-05-27 00:00:00
retrieving news from: 2021-05-26 00:00:00
retrieving news from: 2021-05-25 00:00:00
retrieving news from: 2021-05-24 00:00:00
retrieving news from: 2021-05-23 00:00:00
retrieving news from: 2021-05-22 00:00:00
retrieving news from: 2021-05-21 00:00:00
retrieving news from: 2021-05-20 00:00:00
retrieving news from: 2021-05-19 00:00:00
retrieving news from: 2021-05-18 00:00:00
retrieving news f

In [9]:
# Get third topic
economy_headlines, _ = get_headlines("economy")


Fetching news about 'economy'
******************************
retrieving news from: 2021-06-08 00:00:00
retrieving news from: 2021-06-07 00:00:00
retrieving news from: 2021-06-06 00:00:00
retrieving news from: 2021-06-05 00:00:00
retrieving news from: 2021-06-04 00:00:00
retrieving news from: 2021-06-03 00:00:00
retrieving news from: 2021-06-02 00:00:00
retrieving news from: 2021-06-01 00:00:00
retrieving news from: 2021-05-31 00:00:00
retrieving news from: 2021-05-30 00:00:00
retrieving news from: 2021-05-29 00:00:00
retrieving news from: 2021-05-28 00:00:00
retrieving news from: 2021-05-27 00:00:00
retrieving news from: 2021-05-26 00:00:00
retrieving news from: 2021-05-25 00:00:00
retrieving news from: 2021-05-24 00:00:00
retrieving news from: 2021-05-23 00:00:00
retrieving news from: 2021-05-22 00:00:00
retrieving news from: 2021-05-21 00:00:00
retrieving news from: 2021-05-20 00:00:00
retrieving news from: 2021-05-19 00:00:00
retrieving news from: 2021-05-18 00:00:00
retrieving news

In [10]:
# Instantiate SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [11]:
# Create function that computes average compound sentiment of headlines for each day
def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment

In [12]:
# Get averages of each topics sentiment
aapl_avg = headline_sentiment_summarizer_avg(aapl_headlines)
trade_avg = headline_sentiment_summarizer_avg(trade_headlines)
economy_avg = headline_sentiment_summarizer_avg(economy_headlines)


In [13]:
# Combine Sentiment Averages into DataFrame
topic_sentiments = pd.DataFrame(
    {
        "aapl_avg": aapl_avg,
        "trade_avg": trade_avg,
        "economy_avg": economy_avg,
    }
)

In [14]:
# Set the index value of the sentiment averages DataFrame to be the series of dates.
topic_sentiments.index = pd.to_datetime(dates)

In [15]:
# Merge with AAPL returns
topic_sentiments = aapl_returns.join(topic_sentiments).dropna(how="any")

# Display data
display(topic_sentiments)

,close,aapl_avg,trade_avg,economy_avg
2021-05-11,-0.007962,0.174040,0.224295,0.131245
2021-05-12,-0.024076,0.047090,-0.029085,-0.065995
2021-05-13,0.017262,0.001220,0.036290,-0.084570
2021-05-14,0.019609,0.055005,-0.008860,-0.084060
2021-05-17,-0.008870,0.039880,-0.017345,-0.013745
2021-05-18,-0.011167,0.013080,0.042350,0.184525
2021-05-19,-0.001282,0.072925,0.113970,0.046040
2021-05-20,0.021012,0.165165,-0.024540,-0.034870
2021-05-21,-0.014767,0.017100,0.089625,0.092740
2021-05-24,0.013314,0.167080,-0.052130,0.078570


In [16]:
# Correlate the headlines' sentiment to returns
topic_sentiments.corr().style.background_gradient()

,close,aapl_avg,trade_avg,economy_avg
close,1.000000,0.199874,-0.174067,-0.423835
aapl_avg,0.199874,1.000000,-0.071164,-0.094994
trade_avg,-0.174067,-0.071164,1.000000,0.436567
economy_avg,-0.423835,-0.094994,0.436567,1.000000
